In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [3]:
df = pd.read_csv("../data/features_15k.csv")

X = df.drop(columns=["image_id", "label"])
y = df["label"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

model = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

print("Model trained for RL threshold learning")


Model trained for RL threshold learning


In [4]:
probs = model.predict_proba(X_val)
confidences = probs.max(axis=1)
preds = probs.argmax(axis=1)


In [5]:
CONF_BINS = 20           # discretize confidence
ACTIONS = [-0.05, 0.0, 0.05]  # decrease, keep, increase threshold

Q = np.zeros((CONF_BINS, len(ACTIONS)))

alpha = 0.1    # learning rate
gamma = 0.9    # discount factor
epsilon = 0.2  # exploration


In [6]:
def reward_fn(conf, pred, true, threshold):
    if conf < threshold:
        return -0.2    # small penalty for abstaining
    elif pred == true:
        return +1.0    # reward correct confident prediction
    else:
        return -1.0    # heavy penalty for wrong confident prediction


In [7]:
threshold = 0.5
episodes = 50

for ep in range(episodes):
    for i in range(len(confidences)):
        state = min(int(confidences[i] * CONF_BINS), CONF_BINS - 1)

        # ε-greedy action selection
        if np.random.rand() < epsilon:
            action_idx = np.random.randint(len(ACTIONS))
        else:
            action_idx = np.argmax(Q[state])

        action = ACTIONS[action_idx]
        new_threshold = np.clip(threshold + action, 0.1, 0.9)

        r = reward_fn(
            confidences[i],
            preds[i],
            y_val.iloc[i],
            new_threshold
        )

        next_state = state

        Q[state, action_idx] += alpha * (
            r + gamma * np.max(Q[next_state]) - Q[state, action_idx]
        )

        threshold = new_threshold

    if ep % 10 == 0:
        print(f"Episode {ep}, threshold = {threshold:.3f}")


Episode 0, threshold = 0.100
Episode 10, threshold = 0.450
Episode 20, threshold = 0.650
Episode 30, threshold = 0.200
Episode 40, threshold = 0.150


In [8]:
print("✅ Optimized confidence threshold:", round(threshold, 3))


✅ Optimized confidence threshold: 0.1


In [9]:
correct = 0
confident = 0

for i in range(len(confidences)):
    if confidences[i] >= threshold:
        confident += 1
        if preds[i] == y_val.iloc[i]:
            correct += 1

print("Confident accuracy:", correct / confident)
print("Coverage:", confident / len(confidences))


Confident accuracy: 0.7284444444444444
Coverage: 1.0


In [10]:
with open("../backend/threshold.txt", "w") as f:
    f.write(str(0.68))  # <-- use your optimized threshold


In [11]:
import numpy as np

# Get probabilities on validation set
probs = binary_model.predict_proba(X_test)
conf = probs.max(axis=1)
preds = probs.argmax(axis=1)   # 0 = REAL, 1 = FAKE
true = y_test.values

def compute_reward(threshold):
    total_reward = 0

    for i in range(len(true)):
        confidence = conf[i]
        pred = preds[i]
        actual = true[i]

        # Decision rule
        if pred == 1 and confidence >= threshold:
            decision = 1   # FAKE
        else:
            decision = 0   # REAL

        # Reward logic
        if decision == actual:
            total_reward += 1
        else:
            if decision == 1 and actual == 0:
                total_reward -= 3   # False FAKE (high penalty)
            else:
                total_reward -= 1   # False REAL

    return total_reward


# Try multiple thresholds
thresholds = np.arange(0.50, 0.90, 0.02)
rewards = []

for t in thresholds:
    r = compute_reward(t)
    rewards.append(r)

best_threshold = thresholds[np.argmax(rewards)]

print("Optimal Threshold:", round(best_threshold, 2))
print("Max Reward:", max(rewards))


NameError: name 'binary_model' is not defined